In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as r
import pickle as pkl
from types import SimpleNamespace
import os
import time

In [2]:
cwd = os.getcwd()
cwd

'd:\\D_Drive\\Github\\Thesis\\GraphSVX_graph_properties\\graph_theoretic_properties'

In [3]:
data = SimpleNamespace()
with open('../data/BA-2motif.pkl', 'rb') as fin:
    data.edge_index, data.x, data.y = pkl.load(fin)

In [4]:
data.y.shape

(1000, 2)

In [5]:
data.indices = np.array(range(data.edge_index.shape[0]))

In [6]:
# For the duplication
new_data = SimpleNamespace()
new_data.indices = np.repeat(data.indices, 2)
new_data.edge_index = []
new_data.x = []
new_data.y = []

for i in list(range(data.edge_index.shape[0])):
    new_data.edge_index.append(data.edge_index[i])
    new_data.x.append(data.x[i])
    new_data.y.append(data.y[i])

    new_data.edge_index.append(data.edge_index[i])
    new_data.x.append(data.x[i])
    new_data.y.append(data.y[i])

new_data.edge_index = np.array(new_data.edge_index)
new_data.x = np.array(new_data.x)
new_data.y = np.array(new_data.y)

start = time.time()

with open('../data/syn6_identical.pkl', 'wb') as f1:
    pkl.dump([new_data.indices, new_data.edge_index, new_data.x, new_data.y], f1)

print('Time (sec):',time.time() - start)

Time (sec): 0.01715826988220215


In [7]:
# Testing the np.random.permutation
a = np.random.randint(2, size=(4, 4))
c = np.random.randint(2, size=a.shape)
b = np.random.permutation(a)

a+c

array([[1, 1, 0, 1],
       [1, 2, 1, 2],
       [1, 1, 1, 1],
       [1, 1, 2, 1]])

In [8]:
len(new_data.edge_index)

2000

In [9]:
a = np.random.randint(3, size=(4,4))
a = (a != 0)*1
a

array([[0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 1],
       [0, 1, 1, 1]])

In [10]:
pert_data = SimpleNamespace()
pert_data.indices = new_data.indices
pert_data.edge_index = []
pert_data.x = []
pert_data.y = new_data.y
for i in list(range(1,len(pert_data.indices),2)):
    # Perturbing the adjacency matrix
    a = np.random.permutation(new_data.edge_index[i])
    a = a + np.random.randint(2, size=a.shape)
    a = ((a != 0)*1).astype(float)
    pert_data.edge_index.append(a)

    # Perturbing the feature matrix
    b = np.random.permutation(new_data.x[i])
    b = b + np.random.randint(2, size=b.shape)
    b = ((b != 0)*1).astype(float)
    pert_data.x.append(b)

pert_data.edge_index = np.array(pert_data.edge_index)
pert_data.x = np.array(pert_data.x)


In [11]:
start = time.time()

with open('../data/syn6_similar.pkl', 'wb') as f1:
    pkl.dump([new_data.indices, new_data.edge_index, new_data.x, new_data.y], f1)

print('Time (sec):',time.time() - start)

Time (sec): 0.014189720153808594
